In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Parte Exploratoria

In [ ]:
#Carga del dataset

df=pd.read_csv("../input/craigslist-carstrucks-data/vehicles.csv")

In [ ]:
df.head()

In [ ]:
print("El dataset tiene: ",len(df),"filas en el dataset") 

In [ ]:
#tipo de datos en cada columna
df.info()

In [ ]:
#Se revisa si hay valores nulos
df.isnull().sum()

Se puede observar que hay muchos valores nulos en varias columnas y adicional muchas de ellas que no se consideran relavantes para la predicción del precio, como lo son el ID, la URL, VIN, region, entre otras; ante esto se decide eliminarlas del dataset y solo hacer tratamiento de faltantes en las columnas relevantes.

In [ ]:
# Bad Predictors

bad_predictors = [
    'id', 'url', 'region', 'region_url', 'VIN', 'drive', 'size', 'county', 'state', 
    'paint_color', 'image_url', 'description', 'lat', 'long', 'posting_date'
]

df.drop(bad_predictors, axis=1, inplace=True)

In [ ]:
df.isna().sum()

# Primera Parte

### 1.Elimine todos los NaN (1 pt)




In [ ]:
df_dropna = df.dropna().copy()
df_dropna.head()

Se utiliza el paquete de pandas profiling, para analizar el dataset ya tratado. Dentro de los datos que se obtienen, es la matriz de correlacion, la cantidad de resgistros, los tipos de variables y otros hallazgos que son importantes para el modelado correcto

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_dropna)

De lo anterior, se decide realizar varios tratamientos adicionales:
1. Eliminar las líneas en donde el precio sea menor al P10 o este por arriba del P90
2. Eliminar los autos con año menor a 2000
3. Eliminar los autos con odometros menores a 100 km
4. Transformar las variables categoricas
5. Eliminar el modelo,ya que no se encontró una forma fácil de estandarizar


In [ ]:
df_dropna = df_dropna[df_dropna['year'] > 2000]
df_dropna = df_dropna[
    (df_dropna['price'] > df_dropna['price'].quantile(.10)) &
    (df_dropna['price'] < df_dropna['price'].quantile(.90))
]

df_dropna = df_dropna[
    (df_dropna['odometer'] > df_dropna['odometer'].quantile(.05)) &
    (df_dropna['odometer'] < df_dropna['odometer'].quantile(.95))
]

In [ ]:
df_dropna.drop(['model'], axis=1, inplace=True)

In [ ]:
df_dropna['cylinders'] = df_dropna['cylinders'].str.replace(r'[^0-9]', '', regex=True)
df_dropna['cylinders'] = df_dropna['cylinders'].str.replace(r'^\s*$', '1', regex=True)
df_dropna['cylinders'] = df_dropna['cylinders'].astype(int)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

tranf=['manufacturer','condition','fuel','title_status','transmission','type','']

df_dropna[tranf[0]] = ordinal_encoder.fit_transform(df_dropna[[tranf[0]]])
df_dropna[tranf[1]] = ordinal_encoder.fit_transform(df_dropna[[tranf[1]]])
df_dropna[tranf[2]] = ordinal_encoder.fit_transform(df_dropna[[tranf[2]]])
df_dropna[tranf[3]] = ordinal_encoder.fit_transform(df_dropna[[tranf[3]]])
df_dropna[tranf[4]] = ordinal_encoder.fit_transform(df_dropna[[tranf[4]]])
df_dropna[tranf[5]] = ordinal_encoder.fit_transform(df_dropna[[tranf[5]]])

In [ ]:
df_dropna.describe()

In [ ]:
#Se divide el dataset en train y test
X = df_dropna.drop(['price'], axis=1)
y = df_dropna['price']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

### 2. Ajuste un Bosque Aleatorio (5 pts)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
    
model = RandomForestRegressor()
model.fit(X_train, y_train)

### 3.[Extra] Puede utilizar GridSearchCV o RandomizedSearchCV para ajustar los hiperparámetros

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [ ]:
param_grid = [
    {'n_estimators': [10, 50, 100], 'max_features': ['sqrt', 'log2']},
    {'bootstrap': [False], 'n_estimators': [10, 100], 'max_features': [2, 3, 4]},
]


grid_search = GridSearchCV(model, param_grid, cv=5, return_train_score=True)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

print(grid_search.best_score_)

### 4.Muestre sus resultados utilizando las métricas: Mean Absolute Error y Mean Squared Error (3 pts)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error

def PrintMetrics(y, y_pred, title=''):
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    maxerror = max_error(y, y_pred)

    print(title)
    print(f"mean_absolute_error: {mae}")
    print(f"mean_squared_error: {mse}")
    print(f"maxerror: {maxerror}")
    print('\n')

In [ ]:
PrintMetrics(y_test, model.predict(X_test), title='Random Forest Regressor: ')

# Segunda Parte

### 1.Proponga un método para completar los valores vacíos, o justifique porque no hacerlo. (5 pts)

In [ ]:
df_fillna = df.copy()
# Se eliminan las filas en donde hay 3 o mas columnas con NA

df_fillna = df_fillna[df_fillna.isna().sum(axis=1) <= 3]

In [ ]:
df_fillna.isna().sum()

In [ ]:
#Se elimina el modelo nuevamemente.Adicional de las mismas consideraciones en cuanto a precio, odometro y precio
df_fillna.drop(['model'], axis=1, inplace=True)

df_fillna = df_fillna[df_fillna['year'] > 2000]
df_fillna = df_fillna[
    (df_fillna['price'] > df_fillna['price'].quantile(.10)) &
    (df_fillna['price'] < df_fillna['price'].quantile(.90))
]

df_fillna = df_fillna[
    (df_fillna['odometer'] > df_fillna['odometer'].quantile(.05)) &
    (df_fillna['odometer'] < df_fillna['odometer'].quantile(.95))
]

In [ ]:
df_fillna.head()

Para este tratamiento, se utilizara imputacion de la moda para las categoricas; los datos faltantes de las numéricas ya se les dio tratamiento al eliminar los outliers. Se valiadara a la vez que Nan no sea la moda en ninguna de las columnas

In [ ]:
#Se revisan la moda y la media de cada colummna, para ver cual se utilizara
df_mode=df_fillna.mode()
df_median=df_fillna.median()

In [ ]:
df_mode

In [ ]:
df_median

In [ ]:
df_fillna.isna().sum()

In [ ]:
for column in ['manufacturer', 'condition', 'cylinders', 'fuel', 'title_status','transmission','type']:
    df_fillna[column].fillna(df_fillna[column].mode()[0], inplace=True)

In [ ]:
df_fillna.isna().sum()

In [ ]:
tranf=['manufacturer','condition','fuel','title_status','transmission','type','cylinders']

df_fillna[tranf[0]] = ordinal_encoder.fit_transform(df_fillna[[tranf[0]]])
df_fillna[tranf[1]] = ordinal_encoder.fit_transform(df_fillna[[tranf[1]]])
df_fillna[tranf[2]] = ordinal_encoder.fit_transform(df_fillna[[tranf[2]]])
df_fillna[tranf[3]] = ordinal_encoder.fit_transform(df_fillna[[tranf[3]]])
df_fillna[tranf[4]] = ordinal_encoder.fit_transform(df_fillna[[tranf[4]]])
df_fillna[tranf[5]] = ordinal_encoder.fit_transform(df_fillna[[tranf[5]]])
df_fillna[tranf[6]] = ordinal_encoder.fit_transform(df_fillna[[tranf[6]]])

In [ ]:
#Se divide el dataset en train y test
X = df_fillna.drop(['price'], axis=1)
y = df_fillna['price']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

### 2.Ajuste otro Bosque Aleatorio (5 pts)

In [ ]:
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)

### 3.Compare sus nuevos resultados utilizando las métricas: Mean Absolute Error y Mean Squared Error (3 pts)

In [ ]:
PrintMetrics(y_test, model2.predict(X_test), title='Random Forest Regressor Second Part: ')

### 4. Ajuste uno o varios modelos de los vistos en clase para mejorar sus resultados (5 pts)

In [ ]:
#Se utilizará Naive Bayes para realizar otro modelado sobre el dataset en donde se eliminaron los faltantes, ya que se obtuvieron errores menores

#Se divide el dataset en train y test
X = df_dropna.drop(['price'], axis=1)
y = df_dropna['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

In [ ]:
PrintMetrics(y_test, nb_model.predict(X_test), title='Naive Bayes : ')

Se obtiene un resultado mucho peor con respecto al obtenido con Random Forest